In [63]:
import numpy as np
import pandas as pd
import math

# Increase precision for display
pd.set_option('display.precision', 8)


# Derivative at one point using Lagrange polyminal

## Algorithm

- **Input**
  1. **Dataset**: A set of discrete points $(x_i, y_i)$ for $i = 0, \dots, N$.
  2. **Target Point**: The value $c$ where the derivative is required.
  3. **Order of Approximation**: $p$ (the number of points to use for interpolation).

- **Steps**

**1. Point Selection**
   * Identify the $p$ points from the dataset that are closest to the target $c$.
   * Let these selected points be denoted as $\{(x_0, y_0), (x_1, y_1), \dots, (x_{p-1}, y_{p-1})\}$.

**2. Construct Lagrange Polynomial**
   * The function $f(x)$ is approximated by the Lagrange polynomial $P(x)$ of degree $p-1$:
     $$P(x) = \sum_{j=0}^{p-1} y_j L_j(x)$$
   * Where the Lagrange basis polynomials $L_j(x)$ are defined as:
     $$L_j(x) = \prod_{\substack{k=0 \\ k \ne j}}^{p-1} \frac{x - x_k}{x_j - x_k}$$

**3. Compute Derivative of Basis Polynomials**
   * Differentiate $P(x)$ with respect to $x$:
     $$P'(x) = \sum_{j=0}^{p-1} y_j L'_j(x)$$
   * The derivative of the basis polynomial $L'_j(x)$ at $x=c$ is calculated as:
     $$L'_j(c) = L_j(c) \cdot \sum_{\substack{k=0 \\ k \ne j}}^{p-1} \frac{1}{c - x_k}$$
     *(Note: If $c$ coincides with a node $x_j$, specific limits apply to avoid division by zero).*

**4. Calculate Final Derivative**
   * Compute the weighted sum of the derivatives:
     $$f'(c) \approx P'(c) = \sum_{j=0}^{p-1} y_j L'_j(c)$$

- Output
* **Approximate Derivative**: $f'(c)$

In [64]:
def calculate_lagrange_weights_v2(x_vals, c):
    """
    Computes the derivative of Lagrange basis polynomials L'_j(c).
    FIXED: Corrected sign error in node-aligned case.
    """
    n = len(x_vals)
    weights = np.zeros(n)
    
    # Check if c is exactly one of the nodes
    node_index = -1
    for i, x in enumerate(x_vals):
        if math.isclose(x, c, rel_tol=1e-9):
            node_index = i
            break
            
    if node_index != -1:
        # Case 1: c is a node x_k
        k = node_index
        for i in range(n):
            if i == k:
                # Diagonal element: Sum of reciprocals 1/(xk - xj)
                for j in range(n):
                    if j != k:
                        weights[i] += 1.0 / (x_vals[k] - x_vals[j])
            else:
                # Off-diagonal elements: L'_i(xk)
                # Formula: (1 / (xi - xk)) * Product(...)
                product_term = 1.0
                for j in range(n):
                    if j != k and j != i:
                        product_term *= (x_vals[k] - x_vals[j]) / (x_vals[i] - x_vals[j])
                
                # FIXED: Pre-factor is 1/(xi - xk), NOT 1/(xk - xi)
                weights[i] = (1.0 / (x_vals[i] - x_vals[k])) * product_term
    else:
        # Case 2: c is not a node (General case)
        # This part was already correct, but we include it for completeness
        for i in range(n):
            Li_c = 1.0
            for j in range(n):
                if i != j:
                    Li_c *= (c - x_vals[j]) / (x_vals[i] - x_vals[j])
            
            sum_part = 0.0
            for j in range(n):
                if i != j:
                    sum_part += 1.0 / (c - x_vals[j])
            
            weights[i] = Li_c * sum_part
            
    return weights

In [65]:
def solve_derivative_fixed(points, target_c, num_points=3):
    # 1. Select Neighbors
    sorted_by_dist = sorted(points, key=lambda p: abs(p[0] - target_c))
    selected_points = sorted(sorted_by_dist[:num_points], key=lambda p: p[0])
    
    x_subset = np.array([p[0] for p in selected_points])
    y_subset = np.array([p[1] for p in selected_points])
    
    # 2. Calculate Weights (Using Fixed Function)
    weights = calculate_lagrange_weights_v2(x_subset, target_c)
    
    # 3. Compute Derivative
    terms = y_subset * weights
    derivative = np.sum(terms)
    
    # 4. Create DataFrame
    df = pd.DataFrame({
        'x_k': x_subset,
        'y_k': y_subset,
        "Weight L'_k(c)": weights,
        'Term (y_k * W)': terms
    })
    
    return df, derivative

## Result

In [66]:
# --- INPUT DATA FROM PDF PAGE 3 ---
points = [
    (1.0, -0.641), (1.1, -0.498), (1.2, -0.340), (1.3, -0.165), 
    (1.4, 0.028), (1.5, 0.241), (1.6, 0.477), (1.7, 0.737), 
    (1.8, 1.025), (1.9, 1.343), (2.0, 1.695)
]

# --- EXECUTION FOR PDF EXAMPLES ---
target = 1.5



In [67]:
df_result, dy_dx = solve_derivative_fixed(points, target, num_points=3)
df_result

,x_k,y_k,Weight L'_k(c),Term (y_k * W)
0,1.4,0.028,-5.0,-0.140
1,1.5,0.241,0.0,0.000
2,1.6,0.477,5.0,2.385


In [68]:
print(f"Approximate Derivative f'({target}) = {dy_dx:.6f}")

Approximate Derivative f'(1.5) = 2.245000
